# Lab 8 Dynamic Programming

Made & Presented by Bo Tang

In this lab, we will explore how Dynamic Programming solves problems by breaking down the problem simpler subproblems in a recursive manner. The goal is to apply DP to a component combination problem and compare it with a mathematical programming solution.

In [1]:
# install gurobipy first
! pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 21.9 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

### Dynamic Programming Review

Dynamic Programming (DP) is a powerful optimization technique used to solve complex problems by breaking them down into simpler subproblems. Unlike mathematical programming approaches, DP provides a general and versatile framework that can address problems not easily formulated as linear or integer programs. (Many real-life problems often cannot be formulated as linear or mathematical programs.)

In DP, the problem is divided into several stages, each with:
- **Stages** $n = 1, 2, \cdots, N$: A sequence of decision points.
- **State Variables** $s_n$: Describe the system's current state, containing all necessary information to make the next decision.
- **Decision Variables** $x_n$: The choices available at each stage, which affect the next state.
- **Transition Relations** $s_{n+1} = g_n(s_n, x_n)$: Define how the system moves from one state to another based on the decision made.
- **Immediate Return** $h_n(s_n, x_n)$: The reward or cost associated with each decision at each stage.

The solution to a dynamic programming problem typically involves **backward recursion**, where we start from the last stage and move backward to calculate the optimal solution. With these basic concepts, we can express the optimal return from stage $n$ to the end of the horizon as:
$$
f_n(s_n) = \max_{x_n} \big[ h_n(s_n, x_n) + f_{n+1}(g_n(s_n, x_n)) \big]
$$

This process allows us to calculate the optimal solution by working from the end of the problem back to the start.

1. **Initialize the Final Stage**: Start from the last stage, calculating the optimal outcome for each possible state at that stage. This involves setting a base case that defines the final return without further decisions.
   - $f_N(s_N) = \max_{x_N} \big[ h_N(s_N, x_N) \big] \quad \forall s_N$
   
2. **Recursive Calculation for Previous Stages**: Move backward through each stage, using the values computed for future stages recursively to determine the optimal decision at each state. By storing these values, we avoid recalculating results for the same states, increasing efficiency.
   - $f_{N-1}(s_{N-1}) = \max_{x_{N-1}} \big[  h_{N-1}(s_{N-1}, x_{N-1}) + f_{N}(g_{N-1}(s_{N-1}, x_{N-1})) \big] \quad \forall s_{N-1}$
   
3. **Construct the Optimal Solution**: By the time we reach the first stage, we have determined the sequence of optimal decisions and the total objective value, representing the optimal solution for the entire problem.
   - $f_1(s_1) = \max_{x_1} \big[  h_1(s_1, x_1) + f_2(g_1(s_1, x_1)) \big]$

### Recursion Review

Recursion is a problem-solving technique where a function calls itself to solve smaller instances of the same problem. It’s particularly useful when a problem can naturally be divided into similar subproblems. A recursive solution typically has:

1. **Base Case**: This stops the recursion once a condition is met.
2. **Recursive Case**: The function calls itself to solve a smaller subproblem, moving closer to the base case with each call.

In Dynamic Programming, recursion helps to break down a complex problem into simpler subproblems. However, DP often adds **memoization** to recursion, storing intermediate results to avoid redundant calculations.

#### Example Problem: Fibonacci Sequence (LeetCode #509)

The Fibonacci sequence is a classic example of recursion. Each term is the sum of the two preceding terms, with the sequence starting as $0, 1, 1, 2, 3, 5, \ldots$

The recursive formula is:
$$
F(n) = F(n-1) + F(n-2)
$$

Here’s a simple Python solution that uses recursion:

In [3]:
def fibonacci(n):
    # Base cases
    if n == 0:
        return 0
    elif n == 1:
        return 1
    # Recursive case
    return fibonacci(n-1) + fibonacci(n-2)

In [4]:
fibonacci(10)

55

While this solution is straightforward, it’s inefficient for large inputs because it recalculates the same Fibonacci values multiple times.

We can improve the efficiency by storing computed values, known as **memoization**, to avoid redundant calculations.

In [15]:
def fibonacci(n, memo={}):
    # base cases
    if n == 0:
        return 0
    elif n == 1:
        return 1
    # check if value is already computed
    if n in memo:
        return memo[n]
    # recursive calculation with memoization
    memo[n] = fibonacci(n-1, memo) + fibonacci(n-2, memo)
    return memo[n]

In [16]:
fibonacci(10)

55

### Problem Statement

An electronic system consists of four components: A, B, C, and D. The system requires all four components to function, and its reliability can be increased by adding parallel units for one or more of these components. The reliability and cost for different numbers of parallel components are shown in the tables below.

##### Probability of Functioning

| # of Units | Reliability of A | Reliability of B | Reliability of C | Reliability of D |
|------------|------------------|------------------|------------------|------------------|
| 1          | 0.5              | 0.6              | 0.7              | 0.5              |
| 2          | 0.6              | 0.7              | 0.8              | 0.7              |
| 3          | 0.8              | 0.8              | 0.9              | 0.9              |

##### Cost

| # of Units | Cost of A | Cost of B | Cost of C | Cost of D |
|------------|-----------|-----------|-----------|-----------|
| 1          | \$100     | \$200     | \$100     | \$200     |
| 2          | \$200     | \$400     | \$300     | \$300     |
| 3          | \$300     | \$500     | \$400     | \$400     |

##### Goal

Given a budget of $1000, determine how the money should be allocated to maximize the system's reliability.

In [7]:
# parameters
reliabilities = {
    "A": [0, 0.5, 0.6, 0.8],
    "B": [0, 0.6, 0.7, 0.8],
    "C": [0, 0.7, 0.8, 0.9],
    "D": [0, 0.5, 0.7, 0.9]
}
costs = {
    "A": [0, 100, 200, 300],
    "B": [0, 200, 400, 500],
    "C": [0, 100, 300, 400],
    "D": [0, 200, 300, 400]
}

# indices
components = ["A", "B", "C", "D"]
units = [1, 2, 3]

### Mathematical Programming

1. Build an integer linear model formulation. (Hint: $P_A \times P_B \times P_C \times P_D \rightarrow \log(P_A) + \log(P_B) + \log(P_C) + \log(P_D)$.)
2. Solve the model with Gurobi.

In [18]:
# ceate a model
m = gp.Model("System Reliability")
# variables
x = m.addVars(components, units, vtype=GRB.BINARY, name="x")

In [20]:
print("Solutions:")
obj_val = 1
for c, u in x:
    if x[c,u].x >= 1 - 1e-3:
        print("Component {}: {} units.".format(c, u))
        obj_val *= reliabilities[c][u]
print("Maximum Reliability: {:.4f}".format(obj_val))

Solutions:


AttributeError: Index out of range for attribute 'X'

### Dynamic Programming

- Stages $n = A, B, C, D$:
  - Each stage represents a component in the system: A, B, C, and D.
  - For each stage, we decide how many units of the current component to include (1, 2, or 3).

- State Variables $s_n$:
  - $s_n$ is the remaining budget after choosing units for components up to stage $n$.
  - The initial state, $s_1$, is the full budget (\$1,000).

- Decision Variables $x_n$:
  - $x_n$ represents the number of units (1, 2, or 3) chosen for component $n$ (e.g., A, B, C, or D).
  - These decision variables determine both the cost and the reliability contribution of each component.

- Transition Relations $s_{n+1} = s_n - \text{cost}(x_n)$:
  - The transition describes how the budget changes from one stage to the next.
  - For a given $s_n$, choosing $x_n$ reduces the remaining budget based on the cost of the chosen units for component $n$.

- Immediate Return $h_n(s_n, x_n)$:
  - The immediate return at each stage is the reliability of the chosen units for that component, $\text{reliability}(x_n)$.


- Optimal Return $f(s_n, x_n)$
  - Since system reliability is determined by the product of individual component reliabilities, we define the cumulative reliability function $f(s_n, x_n) = \max_{x_n} \big[ h_n(s_n, x_n) \times f(s_{n-1}) \big]$.

Question:

- What value of base case $f(s_0)$ should be set?

1.  Use the table structure below to record each decision and result. Work from Component A to Component D, updating the corresponding optimal choices for each stage.

| Stage / Decsion | 1 Unit | 2 Units | 3 Units |
|-------------------|------------------|--------------------|--------------------|
| **A**             |                  |                    |                    |
| **B**             |                  |                    |                    |
| **C**             |                  |                    |                    |
| **D**             |                  |                    |                    |


2. Use dynamic programming finds the maximize system reliability.

In [5]:
# recursive dynamic programming function
def dp(comp_ind, budget, memo={}):
    # base case: no components left
    if comp_ind >= len(components):
        max_reliability, best_units = 1, []
        return max_reliability, best_units
    # check if solution already computed in memo
    if (comp_ind, budget) in memo:
        return memo[comp_ind, budget]
    # init setting
    max_reliability = 0
    best_units = []
    c = components[comp_ind]
    # recursive case: try all unit options for the current component
    for u in units:
        cost = costs[c][u]
        # check if feasible
        if budget >= cost:
             # Recursive call for the next component
            next_reliability, next_units = dp(comp_ind + 1, budget - cost, memo)
            # Calculate current reliability
            current_reliability = reliabilities[c][u] * next_reliability
            # get best one
            if current_reliability > max_reliability:
                max_reliability = current_reliability
                best_units = [u] + next_units
    # memoize
    # TODO: update memeory
    memo[(comp_ind, budget)] = (max_reliability, best_units)
    # return the result
    return max_reliability, best_units

In [8]:
# solve
max_reliability, unit_configuration = dp(comp_ind=0, budget=1000)
# solution
print("Solutions:")
obj_val = 1
for i, c in enumerate(components):
    print("Component {}: {} units.".format(c, unit_configuration[i]))
print("Maximum Reliability: {:.4f}".format(max_reliability))

Solutions:
Component A: 3 units.
Component B: 1 units.
Component C: 1 units.
Component D: 3 units.
Maximum Reliability: 0.3024
